## Set Up Environment# 

In [1]:
network = "yeouido"  # set this to one of mainnet, yeouido, euljiro, pagoda, or custom
contracts_file_custom = 'contracts_20210106025209.pkl'

connections = {
"mainnet": {"iconservice": "https://ctz.solidwallet.io",       "nid": 1},
"yeouido": {"iconservice": "https://bicon.net.solidwallet.io", "nid": 3},
"euljiro": {"iconservice": "https://test-ctz.solidwallet.io",  "nid": 2},
"pagoda":  {"iconservice": "https://zicon.net.solidwallet.io", "nid":80},
"custom":  {"iconservice": "http://52.53.175.151:9000/",       "nid": 3}}

env = connections[network]

In [2]:
from iconsdk.exception import JSONRPCException
from iconsdk.libs.in_memory_zip import gen_deploy_data_content
from iconsdk.icon_service import IconService
from iconsdk.providers.http_provider import HTTPProvider
from iconsdk.builder.transaction_builder import CallTransactionBuilder, TransactionBuilder, DeployTransactionBuilder
from iconsdk.builder.call_builder import CallBuilder
from iconsdk.signed_transaction import SignedTransaction
from iconsdk.wallet.wallet import KeyWallet
from iconsdk.utils.convert_type import convert_hex_str_to_int
from repeater import retry
from shutil import make_archive
import pickle as pkl
from datetime import datetime
import json
import os

ICX = 1000000000000000000 # 10**18
GOVERNANCE_ADDRESS = "cx0000000000000000000000000000000000000000"
ORACLE = "cx61a36e5d10412e03c907a507d1e8c6c3856d9964"

@retry(JSONRPCException, tries=10, delay=1, back_off=2)
def get_tx_result(_tx_hash):
    tx_result = icon_service.get_transaction_result(_tx_hash)
    return tx_result

In [4]:
contracts = {}

# with open(contracts_file_custom, 'rb') as f:
#     contracts = pkl.load(f)

contracts = {'loans': {'zip': 'core_contracts/loans.zip',
  'SCORE': ''},
 'staking': {'zip': 'core_contracts/staking.zip',
  'SCORE': ''},
 'dividends': {'zip': 'core_contracts/dividends.zip',
  'SCORE': ''},
 'reserve_fund': {'zip': 'core_contracts/reserve_fund.zip',
  'SCORE': ''},
 'rewards': {'zip': 'core_contracts/rewards.zip',
  'SCORE': ''},
 'dex': {'zip': 'core_contracts/dex.zip',
  'SCORE': ''},
 'governance': {'zip': 'core_contracts/governance.zip',
  'SCORE': ''},
 'dummy_oracle': {'zip': 'core_contracts/dummy_oracle.zip',
  'SCORE': ''},
 'sicx': {'zip': 'token_contracts/sicx.zip',
  'SCORE': ''},
 'icd': {'zip': 'token_contracts/icd.zip',
  'SCORE': ''},
 'bal': {'zip': 'token_contracts/bal.zip',
  'SCORE': ''},
 'bwt': {'zip': 'token_contracts/bwt.zip',
  'SCORE': ''}}

In [5]:
icon_service = IconService(HTTPProvider(env["iconservice"], 3))
NID = env["nid"]

In [6]:
wallet = KeyWallet.create()
file_path = "./keystores/keystore_test1.json"
wallet.store(file_path, "test1_Account")
wallet = KeyWallet.load("keystores/keystore_test1.json", "test1_Account")

In [7]:
# Balanced test wallet
bwallet = KeyWallet.create()
file_path = "./keystores/balanced_test.json"
bwallet.store(file_path, "xxxx")
btest_wallet = KeyWallet.load("keystores/balanced_test.json", "xxxx")

In [51]:
wallet.get_address()


'hx020c1a80b5fdcd44a1583c98ae80dc16d19910c1'

In [10]:
test_wallet = KeyWallet.load("./keystores/keystore_test1.json", "test1_Account")
transaction = TransactionBuilder()\
    .from_(wallet.get_address())\
    .to(btest_wallet.get_address())\
    .value(100 * ICX)\
    .step_limit(1000000) \
    .nid(NID) \
    .nonce(2) \
    .version(3) \
    .build()
signed_transaction = SignedTransaction(transaction, test_wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash

'0x3b0a7ebab2e256bdc7f671f208e1c8bce0b88bf0c7cadac93896a2f4d17e7f03'

In [11]:
# Add the contracts. Copy the output from this cell the the one above to save the contract addresses.
# contracts['dividends'] = {'zip': 'core_contracts/dividends.zip', 'SCORE': 'cxceadb6419b92f7170b38b8328dbd8deefb4a1cdb'}
deploy = ['loans', 'staking', 'dividends', 'rewards', 'reserve_fund', 'governance', 'dummy_oracle', 'sicx', 'dex', 'icd', 'bal', 'bwt']
for directory in {"core_contracts", "token_contracts"}:
    with os.scandir(directory) as it:
        for file in it:
            archive_name = directory + "/" + file.name
            if file.is_dir() and file.name in deploy:
                make_archive(archive_name, "zip", directory, file.name)
                contracts[file.name]['zip'] = archive_name + '.zip'

now = datetime.utcnow()
filename = f'contracts_{now.year}{str(now.month).zfill(2)}{str(now.day).zfill(2)}{str(now.hour).zfill(2)}{str(now.minute).zfill(2)}{str(now.second).zfill(2)}.pkl'
print(f'saving contracts list as {filename}')
with open(filename, 'wb') as f:
    pkl.dump(contracts, f)
contracts

saving contracts list as contracts_20210126061416.pkl


{'loans': {'zip': 'core_contracts/loans.zip', 'SCORE': ''},
 'staking': {'zip': 'core_contracts/staking.zip', 'SCORE': ''},
 'dividends': {'zip': 'core_contracts/dividends.zip', 'SCORE': ''},
 'reserve_fund': {'zip': 'core_contracts/reserve_fund.zip', 'SCORE': ''},
 'rewards': {'zip': 'core_contracts/rewards.zip', 'SCORE': ''},
 'dex': {'zip': 'core_contracts/dex.zip', 'SCORE': ''},
 'governance': {'zip': 'core_contracts/governance.zip', 'SCORE': ''},
 'dummy_oracle': {'zip': 'core_contracts/dummy_oracle.zip', 'SCORE': ''},
 'sicx': {'zip': 'token_contracts/sicx.zip', 'SCORE': ''},
 'icd': {'zip': 'token_contracts/icd.zip', 'SCORE': ''},
 'bal': {'zip': 'token_contracts/bal.zip', 'SCORE': ''},
 'bwt': {'zip': 'token_contracts/bwt.zip', 'SCORE': ''}}

In [12]:
step_limit = 4000100000

for score, loc in contracts.items():
    deploy_transaction = DeployTransactionBuilder()\
        .from_(wallet.get_address())\
        .to(GOVERNANCE_ADDRESS)\
        .nid(NID)\
        .nonce(100)\
        .content_type("application/zip")\
        .content(gen_deploy_data_content(loc["zip"]))\
        .build()

#     step_limit = icon_service.estimate_step(deploy_transaction) + 100000
    
    signed_transaction = SignedTransaction(deploy_transaction, wallet, step_limit)
    tx_hash = icon_service.send_transaction(signed_transaction)

    res = get_tx_result(tx_hash)
    contracts[score]["SCORE"] = res.get('scoreAddress', '')
    print(score)
    print(res)

contracts

loans
{'txHash': '0xba72e5e2ce954f0a5198b7c6f2817450fd08999f05581756479421d01b3e9673', 'blockHeight': 12166018, 'blockHash': '0x337c6eea35b11c4373924b4883d739d7492fefcae95d99411255ca63847e6af6', 'txIndex': 1, 'to': 'cx0000000000000000000000000000000000000000', 'scoreAddress': 'cxbcd38b15de8942f3a1422da95e0be894d5bb8b6a', 'stepUsed': 1520082400, 'stepPrice': 10000000000, 'cumulativeStepUsed': 1520082400, 'eventLogs': [], 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\

governance
{'txHash': '0x721d399d7974cff59623094d25c93b9585e0c18a283ee24f731eecead11bd869', 'blockHeight': 12166052, 'blockHash': '0x17fce861820d876aff40a019fc14cc90c3a01626d5ea093bc9c5678eac88222b', 'txIndex': 1, 'to': 'cx0000000000000000000000000000000000000000', 'scoreAddress': 'cxb3362c2cbc70b586f9090cb3242a1f806f629de9', 'stepUsed': 1145421920, 'stepPrice': 10000000000, 'cumulativeStepUsed': 1145421920, 'eventLogs': [], 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00

{'loans': {'zip': 'core_contracts/loans.zip',
  'SCORE': 'cxbcd38b15de8942f3a1422da95e0be894d5bb8b6a'},
 'staking': {'zip': 'core_contracts/staking.zip',
  'SCORE': 'cx7f745a95bceccbc016d93c020b8f1c5eaffeb66b'},
 'dividends': {'zip': 'core_contracts/dividends.zip',
  'SCORE': 'cx8501ba1bddd05ad49c8a0ad39bed914a94832354'},
 'reserve_fund': {'zip': 'core_contracts/reserve_fund.zip',
  'SCORE': 'cx8cc8493e62810d8038714cf7552c0d9d4fc97708'},
 'rewards': {'zip': 'core_contracts/rewards.zip',
  'SCORE': 'cxe405af0ae3441a3a231e345c390ae3586077c54e'},
 'dex': {'zip': 'core_contracts/dex.zip',
  'SCORE': 'cx3c2941d6e3e347cfa569d480e883a4b45a0ee79c'},
 'governance': {'zip': 'core_contracts/governance.zip',
  'SCORE': 'cxb3362c2cbc70b586f9090cb3242a1f806f629de9'},
 'dummy_oracle': {'zip': 'core_contracts/dummy_oracle.zip',
  'SCORE': 'cx4f0f5311aefb92a8fedcd455f8c57bea6ea7c5e9'},
 'sicx': {'zip': 'token_contracts/sicx.zip',
  'SCORE': 'cx83abf730f54ad9072e68470ddb10636760db59ac'},
 'icd': {'zip':

# Update score

In [56]:
contract = 'loans'
update = contracts[contract]['SCORE']
zip_file = contracts[contract]['zip']
deploy_transaction = DeployTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(update)\
    .nid(NID)\
    .nonce(100)\
    .content_type("application/zip")\
    .content(gen_deploy_data_content(zip_file))\
    .build()

# step_limit = icon_service.estimate_step(deploy_transaction) + 100000
step_limit = 3000000000

signed_transaction = SignedTransaction(deploy_transaction, wallet, step_limit)
tx_hash = icon_service.send_transaction(signed_transaction)

get_tx_result(tx_hash)


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': "JSON schema validation error: 'to' has an invalid value"}


JSONRPCException: {'code': -32602, 'message': "JSON schema validation error: 'to' has an invalid value"} (Json rpc error)

In [13]:
settings = [{'contract': 'sicx', 'method': 'setAdmin', 'params':{'_admin': contracts['staking']['SCORE']}},
            {'contract': 'sicx', 'method': 'setStakingAddress', 'params':{'_address': contracts['staking']['SCORE']}},
            {'contract': 'icd', 'method': 'setAdmin', 'params':{'_admin': contracts['loans']['SCORE']}},
            {'contract': 'icd', 'method': 'setOracle', 'params':{'_address': contracts['dummy_oracle']['SCORE'], '_name': 'Dummy'}},
            {'contract': 'bal', 'method': 'setAdmin', 'params':{'_admin': wallet.get_address()}},
            {'contract': 'bal', 'method': 'setOracle', 'params':{'_address': contracts['dex']['SCORE'], '_name': 'BalancedDex'}},
            {'contract': 'bal', 'method': 'mint', 'params':{'_amount': 10 * ICX}},
            {'contract': 'bwt', 'method': 'setAdmin', 'params':{'_admin': contracts['governance']['SCORE']}},
            {'contract': 'staking', 'method': 'setSicxAddress', 'params':{'_address': contracts['sicx']['SCORE']}},
            {'contract': 'reserve_fund', 'method': 'setLoansScore', 'params':{'_address': contracts['loans']['SCORE']}},
            {'contract': 'reserve_fund', 'method': 'setBalnToken', 'params':{'_address': contracts['bal']['SCORE']}},
            {'contract': 'reserve_fund', 'method': 'setSicxToken', 'params':{'_address': contracts['sicx']['SCORE']}},
            {'contract': 'bal', 'method': 'mintTo', 'params':{'_account': contracts['reserve_fund']['SCORE'], '_amount': 1000 * ICX}},
            {'contract': 'dividends', 'method': 'setLoansScore', 'params':{'_address': contracts['loans']['SCORE']}},
#             {'contract': 'rewards', 'method': 'setAdmin', 'params':{'_admin': wallet.get_address()}},
#             {'contract': 'rewards', 'method': 'set_token_address', 'params':{'_address': contracts['bal']['SCORE']}},
#             {'contract': 'dex', 'method': 'setAdmin', 'params':{'_admin': wallet.get_address()}},
            {'contract': 'loans', 'method': 'setAdmin', 'params':{'_admin': wallet.get_address()}},
            {'contract': 'loans', 'method': 'setGovernance', 'params':{'_address': contracts['governance']['SCORE']}},
            {'contract': 'loans', 'method': 'setDividends', 'params':{'_address': contracts['dividends']['SCORE']}},
            {'contract': 'loans', 'method': 'setReserve', 'params':{'_address': contracts['reserve_fund']['SCORE']}},
            {'contract': 'loans', 'method': 'setRewards', 'params':{'_address': contracts['rewards']['SCORE']}},
            {'contract': 'loans', 'method': 'addAsset', 'params':{'_token_address': contracts['sicx']['SCORE'], 'is_active': 1, 'is_collateral': 1}},
            {'contract': 'loans', 'method': 'addAsset', 'params':{'_token_address': contracts['icd']['SCORE'], 'is_active': 1, 'is_collateral': 0}},
            {'contract': 'loans', 'method': 'addAsset', 'params':{'_token_address': contracts['bal']['SCORE'], 'is_active': 0, 'is_collateral': 1}},
            {'contract': 'governance', 'method': 'setLoansScore', 'params':{'_address': contracts['loans']['SCORE']}},
            {'contract': 'governance', 'method': 'launchBalanced', 'params':{}}]

In [14]:
for sett in settings:
    print(f'\n')
    print(f'Calling {sett["method"]}, with parameters {sett["params"]} on the {sett["contract"]} contract.')
    transaction = CallTransactionBuilder()\
        .from_(wallet.get_address())\
        .to(contracts[sett['contract']]['SCORE'])\
        .value(0)\
        .step_limit(10000000)\
        .nid(NID)\
        .nonce(100)\
        .method(sett['method'])\
        .params(sett['params'])\
        .build()
    signed_transaction = SignedTransaction(transaction, wallet)
    tx_hash = icon_service.send_transaction(signed_transaction)
    tx_hash
    print(get_tx_result(tx_hash))



Calling setAdmin, with parameters {'_admin': 'cx7f745a95bceccbc016d93c020b8f1c5eaffeb66b'} on the sicx contract.
{'txHash': '0x22f36dedea99e4a11484056af9be613d9fcda03ddf1fd67cab36d797603c78ac', 'blockHeight': 12166560, 'blockHash': '0x16bcb1a0ee9b04870d77d11220c71047980d1033957f197ea7b675c7abefaf16', 'txIndex': 1, 'to': 'cx83abf730f54ad9072e68470ddb10636760db59ac', 'stepUsed': 148920, 'stepPrice': 10000000000, 'cumulativeStepUsed': 148920, 'eventLogs': [], 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x

{'txHash': '0x5b062f7e23ddbae0b854d92c20151ddf1fb70aae09914c945c57ef30d0dfa557', 'blockHeight': 12166594, 'blockHash': '0x0b56eef6816cf682be9fdb660d3d3fc49dcde0a9ce6b8d383341afdc48d610e5', 'txIndex': 1, 'to': 'cx0a513d56baf6b5cd2af695cdd0f47006ab592cc1', 'stepUsed': 155900, 'stepPrice': 10000000000, 'cumulativeStepUsed': 155900, 'eventLogs': [{'scoreAddress': 'cx0a513d56baf6b5cd2af695cdd0f47006ab592cc1', 'indexed': ['Transfer(Address,Address,int,bytes)', 'cx0a513d56baf6b5cd2af695cdd0f47006ab592cc1', 'hx020c1a80b5fdcd44a1583c98ae80dc16d19910c1', '0x8ac7230489e80000'], 'data': ['0x4e6f6e65']}, {'scoreAddress': 'cx0a513d56baf6b5cd2af695cdd0f47006ab592cc1', 'indexed': ['Mint(Address,int,bytes)', 'hx020c1a80b5fdcd44a1583c98ae80dc16d19910c1'], 'data': ['0x8ac7230489e80000', '0x4e6f6e65']}], 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00 \x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x08\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00

{'txHash': '0xefafafe0c7f186a6f1c93d982aeb6bf8bbb5f730c263238e6bbf010b2cf939e1', 'blockHeight': 12166629, 'blockHash': '0x976a24c267d3c1cd5cd78c9ecfe232fba06161ca19538faaa03e3a626be92448', 'txIndex': 1, 'to': 'cx0a513d56baf6b5cd2af695cdd0f47006ab592cc1', 'stepUsed': 195780, 'stepPrice': 10000000000, 'cumulativeStepUsed': 195780, 'eventLogs': [{'scoreAddress': 'cx0a513d56baf6b5cd2af695cdd0f47006ab592cc1', 'indexed': ['Transfer(Address,Address,int,bytes)', 'cx0a513d56baf6b5cd2af695cdd0f47006ab592cc1', 'cx8cc8493e62810d8038714cf7552c0d9d4fc97708', '0x3635c9adc5dea00000'], 'data': ['0x4e6f6e65']}, {'scoreAddress': 'cx0a513d56baf6b5cd2af695cdd0f47006ab592cc1', 'indexed': ['Mint(Address,int,bytes)', 'cx8cc8493e62810d8038714cf7552c0d9d4fc97708'], 'data': ['0x3635c9adc5dea00000', '0x4e6f6e65']}], 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00 \x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x08\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00

{'txHash': '0xbb317da047d38b9a2657360ea6206b11fe600e896d1323f3b7a6a42b8958f28b', 'blockHeight': 12166665, 'blockHash': '0xf0cd72a0ee5c64721beb3554cd79b8b522823d97d24f90f2dbb008d4f093a984', 'txIndex': 1, 'to': 'cxbcd38b15de8942f3a1422da95e0be894d5bb8b6a', 'stepUsed': 149720, 'stepPrice': 10000000000, 'cumulativeStepUsed': 149720, 'eventLogs': [], 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\

{'txHash': '0x68b4435af4f8c7c1573ab91a25b508348dd711a36fc028f75ee7634c0049c413', 'blockHeight': 12166693, 'blockHash': '0x09b398ae3b1549229ad88453e446749decb192114cf0b360954f3a4f29feeb87', 'txIndex': 1, 'to': 'cxb3362c2cbc70b586f9090cb3242a1f806f629de9', 'stepUsed': 183760, 'stepPrice': 10000000000, 'cumulativeStepUsed': 183760, 'eventLogs': [], 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\

#  Deposit ICX and Mint ICD Loan

In [15]:
data = "{\"method\": \"_deposit_and_borrow\", \"params\": {\"_asset\": \"ICD\", \"_amount\": 4000000000000000000}}".encode("utf-8")
params = {'_to': contracts['loans']['SCORE'], '_data': data}
transaction = CallTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(contracts['staking']['SCORE'])\
    .value(80 * ICX)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("addCollateral")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash
print(get_tx_result(tx_hash))

{'txHash': '0x8b6bc605b59dd799b7e28254b7e70075672193799cc922dd9e108dfd8798409f', 'blockHeight': 12167127, 'blockHash': '0x1e60f2aec093c4236b4b4919fad218eca104f3f82ff56add65b02ff47041e687', 'txIndex': 1, 'to': 'cx7f745a95bceccbc016d93c020b8f1c5eaffeb66b', 'stepUsed': 671860, 'stepPrice': 10000000000, 'cumulativeStepUsed': 671860, 'eventLogs': [{'scoreAddress': 'cxbcd38b15de8942f3a1422da95e0be894d5bb8b6a', 'indexed': ['CollateralReceived(Address,str)', 'hx020c1a80b5fdcd44a1583c98ae80dc16d19910c1', 'sICX'], 'data': []}, {'scoreAddress': 'cx6c9fd94d581d87124c2a2df598b138d19674f92b', 'indexed': ['OraclePrice(str,str,Address,int)', 'USDICX', 'Dummy', 'cx4f0f5311aefb92a8fedcd455f8c57bea6ea7c5e9'], 'data': ['0x2454d8dcb2cf44bb']}, {'scoreAddress': 'cxbcd38b15de8942f3a1422da95e0be894d5bb8b6a', 'indexed': ['OriginateLoan(Address,str,int,str)', 'hx020c1a80b5fdcd44a1583c98ae80dc16d19910c1', 'ICD', '0x3782dace9d900000'], 'data': ['Loan of 4.0 ICD from Balanced.']}, {'scoreAddress': 'cx6c9fd94d581d8

# readonly method

In [16]:
params = {"_owner": wallet.get_address()}
call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts['loans']['SCORE'])\
                    .method('getAccountPositions')\
                    .params(params)\
                    .build()
result = icon_service.call(call)
result
# int(result, 0)

{'created': '0x5b9c815f2ba18',
 'address': 'hx020c1a80b5fdcd44a1583c98ae80dc16d19910c1',
 'assets': {'sICX': '0x4563918244f400000', 'ICD': '0x3810f6985c940000'},
 'standing': 'Liquidate'}

# test_wallet

In [36]:
# Deposit collateral to fresh wallet
data = "{\"method\": \"_deposit_and_borrow\", \"params\": {\"_asset\": \"ICD\", \"_amount\": 2000000000000000000}}".encode("utf-8")
params = {'_to': contracts['loans']['SCORE'], '_data': data}
transaction = CallTransactionBuilder()\
    .from_(btest_wallet.get_address())\
    .to(contracts['staking']['SCORE'])\
    .value(20 * ICX)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("addCollateral")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, btest_wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash
print(get_tx_result(tx_hash))

{'txHash': '0x5a273aa0f1d39b2818450926cebe7631878caa2d337207dca318a30649d0abf7', 'blockHeight': 12172557, 'blockHash': '0x552669f0cabc86c6f5764d4a76407454a33a1527de6ad0760e240b8ac9075219', 'txIndex': 1, 'to': 'cx7f745a95bceccbc016d93c020b8f1c5eaffeb66b', 'stepUsed': 647480, 'stepPrice': 10000000000, 'cumulativeStepUsed': 647480, 'eventLogs': [{'scoreAddress': 'cxbcd38b15de8942f3a1422da95e0be894d5bb8b6a', 'indexed': ['CollateralReceived(Address,str)', 'hx5112dd0b5ca8f2a67c4fd017f3fbf208fd823831', 'sICX'], 'data': []}, {'scoreAddress': 'cx6c9fd94d581d87124c2a2df598b138d19674f92b', 'indexed': ['OraclePrice(str,str,Address,int)', 'USDICX', 'Dummy', 'cx4f0f5311aefb92a8fedcd455f8c57bea6ea7c5e9'], 'data': ['0x2454d8dcb2cf44bb']}, {'scoreAddress': 'cxbcd38b15de8942f3a1422da95e0be894d5bb8b6a', 'indexed': ['OriginateLoan(Address,str,int,str)', 'hx5112dd0b5ca8f2a67c4fd017f3fbf208fd823831', 'ICD', '0x1bc16d674ec80000'], 'data': ['Loan of 2.0 ICD from Balanced.']}, {'scoreAddress': 'cx6c9fd94d581d8

In [57]:
params = {'_owner': btest_wallet.get_address()}
# params = {}
call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts['loans']['SCORE'])\
                    .method('getAccountPositions')\
                    .params(params)\
                    .build()
result = icon_service.call(call)
result
# int(result, 0)//10**18

{'created': '0x5b9c81c67fd18',
 'address': 'hx5112dd0b5ca8f2a67c4fd017f3fbf208fd823831',
 'assets': {'sICX': '0x1a055690d9db80000', 'ICD': '0x1c087b4c2e4a0000'},
 'standing': 'Liquidate'}

In [89]:
params = {'_owner': btest_wallet.get_address()}
transaction = CallTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(contracts['loans']['SCORE'])\
    .value(0)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("liquidate")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash
print(get_tx_result(tx_hash))

{'txHash': '0x61ccc2cf62d7c4e3cc2a84642670962f92ff1e4741debf2bf98b46d6c191077d', 'blockHeight': 12132241, 'blockHash': '0xe61eb6c7e8e662fdd6098d38ce8a8b7f88be0bd1398855dc36dde490ef8a664a', 'txIndex': 1, 'to': 'cx9803db52f1839dc706da184dc284b07aa0a26fba', 'stepUsed': 336640, 'stepPrice': 10000000000, 'cumulativeStepUsed': 336640, 'eventLogs': [{'scoreAddress': 'cx0a359a96e85a243540b0c8af1c378c4db6f1f766', 'indexed': ['OraclePrice(str,str,Address,int)', 'USDICX', 'Dummy', 'cxe028b74c2ed7ac2a20960d97a73a3b4fa1463dd8'], 'data': ['0x2454d8dcb2cf44bb']}, {'scoreAddress': 'cx9803db52f1839dc706da184dc284b07aa0a26fba', 'indexed': ['PositionStanding(Address,str,str,str)', 'hx15a3a0b62c41663e1a9f8a06d6002bb15cee3957', 'Mining'], 'data': ['572.966145', 'Position up to date.']}], 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x02\x02 \x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x80\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x

In [41]:
params = {'_owner': btest_wallet.get_address()}
call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts['icd']['SCORE'])\
                    .method('balanceOf')\
                    .params(params)\
                    .build()
result = icon_service.call(call)
int(result, 0) // 10**18

2

# repay icd

In [42]:
data = "{\"method\": \"_repay_loan\", \"params\": {}}".encode("utf-8")
params = {'_to': contracts['loans']['SCORE'], '_value': 1 * ICX, '_data': data}
transaction = CallTransactionBuilder()\
    .from_(btest_wallet.get_address())\
    .to(contracts['icd']['SCORE'])\
    .value(0)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("transfer")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, btest_wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash
print(get_tx_result(tx_hash))

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


Retrying in 1 seconds...
{'txHash': '0xdf16ecc2ee27c231bc4630f18e30bba79a49da09beac1a2ebfb6ca015c90e365', 'blockHeight': 12172652, 'blockHash': '0x9e184b0743a4d91764e92db14790d9afdc37111d7e3d4a66f51677d2b2424cac', 'txIndex': 1, 'to': 'cx6c9fd94d581d87124c2a2df598b138d19674f92b', 'stepUsed': 278000, 'stepPrice': 10000000000, 'cumulativeStepUsed': 278000, 'eventLogs': [], 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00

# Withdraw Some Collateral

In [48]:
params = {'_value': 5 * ICX}
transaction = CallTransactionBuilder()\
    .from_(btest_wallet.get_address())\
    .to(contracts['loans']['SCORE'])\
    .value(0)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("withdrawCollateral")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, btest_wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash
print(get_tx_result(tx_hash))

{'txHash': '0x848450bb1062d09607d86ea73d2ed4ec9f6fc758465a71954fc47daaf02125d1', 'blockHeight': 12173737, 'blockHash': '0x72aa1aedc7ebb744a15f3528bf40c66a92608ef1eb6ac4712507bb994949f721', 'txIndex': 1, 'to': 'cxbcd38b15de8942f3a1422da95e0be894d5bb8b6a', 'stepUsed': 299560, 'stepPrice': 10000000000, 'cumulativeStepUsed': 299560, 'eventLogs': [{'scoreAddress': 'cx6c9fd94d581d87124c2a2df598b138d19674f92b', 'indexed': ['OraclePrice(str,str,Address,int)', 'USDICX', 'Dummy', 'cx4f0f5311aefb92a8fedcd455f8c57bea6ea7c5e9'], 'data': ['0x2454d8dcb2cf44bb']}, {'scoreAddress': 'cx83abf730f54ad9072e68470ddb10636760db59ac', 'indexed': ['Transfer(Address,Address,int,bytes)', 'cxbcd38b15de8942f3a1422da95e0be894d5bb8b6a', 'hx5112dd0b5ca8f2a67c4fd017f3fbf208fd823831', '0x4563918244f40000'], 'data': ['0x4e6f6e65']}, {'scoreAddress': 'cxbcd38b15de8942f3a1422da95e0be894d5bb8b6a', 'indexed': ['TokenTransfer(Address,int,str)', 'hx5112dd0b5ca8f2a67c4fd017f3fbf208fd823831', '0x4563918244f40000'], 'data': ['Col

In [59]:
# params = {'_owner': btest_wallet.get_address()}
params = {}
call = CallBuilder().from_(btest_wallet.get_address())\
                    .to(contracts['loans']['SCORE'])\
                    .method('getTotalCollateral')\
                    .params(params)\
                    .build()
result = icon_service.call(call)
result
int(result, 0)//10**18

Raised JSONRPCException while returning the custom response. Error message: {'code': -32005, 'message': 'The event log can not be recorded on readonly context'}


JSONRPCException: {'code': -32005, 'message': 'The event log can not be recorded on readonly context'} (Json rpc error)

# thresholdCheck

In [58]:
params = {'_owner':btest_wallet.get_address() }
transaction = CallTransactionBuilder()\
    .from_(btest_wallet.get_address())\
    .to(contracts['loans']['SCORE'])\
    .value(0)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("thresholdCheck")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, btest_wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash
print(get_tx_result(tx_hash))

{'txHash': '0x79c0d88194154d7ceda64bcd99bdfec464667cf0562a15d0f852bc05ba4e14a9', 'blockHeight': 12174509, 'blockHash': '0x58c87bb30ac01819541e98a039f52e3f2bdb0da39873814c3c82ee39d2c758f6', 'txIndex': 2, 'to': 'cxbcd38b15de8942f3a1422da95e0be894d5bb8b6a', 'stepUsed': 257240, 'stepPrice': 10000000000, 'cumulativeStepUsed': 714120, 'eventLogs': [{'scoreAddress': 'cx6c9fd94d581d87124c2a2df598b138d19674f92b', 'indexed': ['OraclePrice(str,str,Address,int)', 'USDICX', 'Dummy', 'cx4f0f5311aefb92a8fedcd455f8c57bea6ea7c5e9'], 'data': ['0x2454d8dcb2cf44bb']}], 'logsBloom': b'\x00\x00\x00\x00\x00 \x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00 \x00\x00\x00\x10\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00@\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x